<a href="https://colab.research.google.com/github/reggy0/disaster_message_classification/blob/main/Project/compare_word_2_vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare word2vec and word counting
## performing measurement
### Disaster Tweet Classification with Disaster Tweets

------------------
### GOAL
- comparing the performance of different word representation
    - word2vec vs. word count
- Predicting whether a given tweet is about a real disaster or not
- if so, predict a `1`. if not, predict a `0`


### Reference
- [competition main page](https://www.kaggle.com/c/nlp-getting-started)
- [word2vec code](https://www.kaggle.com/slatawa/simple-implementation-of-word2vec)
- [word2vec-implementation-for-beginner](https://www.kaggle.com/manavkapadnis/nlp-word2vec-implementation-for-beginner)
- [word count](https://www.kaggle.com/datarohitingole/disaster-tweet-classification-ridgeclassifiercv)
- [comparing the performance of different Machine Learning Algorithm](https://dibyendudeb.com/comparing-machine-learning-algorithms/)
- [Text Classification with NLP: Tf-Idf vs Word2Vec vs BERT](https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794)

# 0. Importing Libraries

In [ ]:
# for loading and preprocessing the data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

# for training the model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import tree, linear_model, neighbors, naive_bayes, ensemble
#! pip3 install xgboost
from xgboost import XGBClassifier
#! pip3 install catboost
from catboost import CatBoostClassifier

# for evaluating classification model
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

# for data cleaning
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')


# Keras
#! pip3 install tensorflow
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM

# for word2ve
#! pip3 install gensim
#! pip3 install konlpy
# import gensim
# from gensim.models import Word2Vec

# Comparing all machine learning algorithms
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve

ModuleNotFoundError: No module named 'xgboost'

# 2. Data Preprocessing
## Contents
### 1. Clean the data
- dealing with missing values
- replace some commonly occuring shorthands
- remove any characters other then alphabets
- convert all dicitonary to lower case(for consistency)
- lemmatize

### 2-1. word tokenization and word2vec

### 2-2. Convert text to vectors using Counter vectorizer





## Data Description
--------------
### Files
- `train.csv` : the training set
- `test.csv` : the test set
- `sample_submission.csv` : a sample submission file in the correct format

### Columns
- `id` : a unique identifier for each tweet
- `text` : the text of the tweet
- `location` : the location the tweet was sent from 
- `keyword` : a particular keyword from th tweet
- `target` : in train.csv only, this denotes whether a tweet is about a real disaster(1) or not(0)

In [ ]:
# loading the data set
data_path = './TFIDFInput/'
train = pd.read_csv(data_path + 'train_added.csv', encoding='CP949')

In [ ]:
print('train_shape:', train.shape)
train.head()

In [ ]:
# split the data <train : test = 8 : 2>
train, test = train_test_split(train, test_size = 0.20, random_state = 0)

In [ ]:
print('train shape:', train.shape)
train.head()

In [ ]:
print('test shape:', test.shape)
test.head()

## 1. Clean the data
#### (1) Dealing with Missing Values

In [ ]:
all_data = [train,test]
for data in all_data:
    data.drop(["location", "id"], axis = 1, inplace = True)

In [ ]:
# data prepocessing with regrex

def remove_URL(text): # remove url pattern in text
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text): # remove html pattern in text
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return html.sub(r'', text)
    #return re.sub(html, '', text)

def remove_punct(text): # remove punctuation in text: (;, ', ", :, ., , etc..)
  table = str.maketrans('', '', string.punctuation)
  return text.translate(table)

#### (2) Replace some commonly occuring shorthands

In [ ]:
for data in all_data:
  data['text'] = data['text'].apply(lambda x: remove_URL(x))
  data['text'] = data['text'].apply(lambda x: remove_html(x))
  data['text'] = data['text'].apply(lambda x: remove_punct(x))

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"you'll", "you will", text)
    text = re.sub(r"i'll", "i will", text)
    text = re.sub(r"she'll", "she will", text)
    text = re.sub(r"he'll", "he will", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"here's", "here is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"shouldn't", "should not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    return text

In [ ]:
train['clean_text'] = train['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

In [ ]:
test.head()

#### (3) remove any characters other then alphabets
#### (4) convert all dicitonary to lower case(for consistency)
#### (5) lemmatize

In [ ]:
def massage_text(text):
    ## remove anything other then characters and put everything in lowercase
    tweet = re.sub("[^a-zA-Z]", ' ', text) # (3)
    tweet = tweet.lower() # (4) 
    tweet = tweet.split()

    lem = WordNetLemmatizer() # (5)
    tweet = [lem.lemmatize(word) for word in tweet
             if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    return tweet
    print('--here goes nothing')
    print(text)
    print(tweet)

train['clean_text'] = train['text'].apply(massage_text)
test['clean_text'] = test['text'].apply(massage_text)

In [ ]:
train.iloc[0:10][['text','clean_text']]

## 2-1. Word Tokenization and word2vec
- tokenize the clean text column

In [ ]:
train['tokens'] = train['clean_text'].apply(lambda x: word_tokenize(x))
test['tokens'] = test['clean_text'].apply(lambda x: word_tokenize(x))

In [ ]:
train.head()

#### convert our data(words) into vectors

In [ ]:
# #first, create a list corpus which we would be using to train word2vec mappings
# def fn_pre_process_data(doc):
#     for rec in doc:
#         yield gensim.utils.simple_preprocess(rec)

# corpus = list(fn_pre_process_data(train['clean_text']))
# corpus += list(fn_pre_process_data(test['clean_text']))

In [ ]:
# #inititate the embedding model, we will come back to the passed arguments later
# print('initiated ...')
# wv_model = Word2Vec(corpus,vector_size=150,window=3,min_count=2)
# #wv_model.build_vocab(corpus)
# wv_model.train(corpus,total_examples=len(corpus),epochs=10)
# wv_model.save(data_path + 'word2vec.model')

In [ ]:
# # convert the train and text tokens
# def get_word_embeddings(token_list,vector,k=150):
#     if len(token_list) < 1:
#         return np.zeros(k)
#     else:

#         vectorized = [vector.wv[word] if word in vector.wv else np.random.rand(k) for word in token_list] 
    
#     sum = np.sum(vectorized,axis=0)
#     ## return the average
#     return sum/len(vectorized)       

# def get_embeddings(tokens,vector):
#         embeddings = tokens.apply(lambda x: get_word_embeddings(x, wv_model))
#         return list(embeddings)

In [ ]:
# train_embeddings = get_embeddings(train['tokens'],wv_model)
# test_embeddings = get_embeddings(test['tokens'],wv_model)

In [ ]:
# model_path = './model/'
# wv_model.save(model_path + 'word2vec_1.model')

## 2-2. Convert text to vectors using Counter vectorizer

### What is the Count Vectorizer?
- convert a collection of text documents to a matrix of token counts

### How to Use
```python
# python example code
corpus = ["This is the first document", "This document is the second document", "And this is the thrid one"]
vectorize = CounterVectorize()
X = vectorize.fit_transform(corpus)
```
- vectorizer.get_feature_names_out()
> array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this'], ...)
- X.toarray()
> [[0 1 1 1 0 0 1 0 1]  
 [0 2 0 1 0 1 1 0 1]  
 [1 0 0 1 1 0 1 1 1]]

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_features=2000)

X = count_vectorizer.fit_transform(train["clean_text"]).toarray()
test_tmp = count_vectorizer.transform(test["clean_text"]).toarray()
y = train['target']

In [ ]:
X_train = X
X_test = test_tmp
y_train = train['target']
y_test = test['target']

# 3. Model
## Contents
- train the model
    - RidgeClassifierCV
    - sgd classifier
    - BernoulliNB 
    - RandomForest

## Model Description
--------------
### Ensemble
- Combine the predictions of several base estimators built with a given learning algorithm 
    - in order to improve generalizability / robustness over a single estimator.
- Boosting of Ensemble types

### Performance - f1-score


## 3-2. Train the model

In [ ]:
MLA = [
    #Ensemble Methods
    ensemble.RandomForestClassifier(),
    
    #GLM
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB()
    ]

# Comapring all MLA
## word2vec
- precision
- recall
- accuracy
- f1-score

In [ ]:
train_embeddings = np.array(X_train)
test_embeddings = np.array(X_test)

In [ ]:
# Comparing all machine learning algorithms
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve

row_index = 0
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)

for alg in MLA:
    predicted = alg.fit(train_embeddings, y_train).predict(test_embeddings)
    fp, tp, th = roc_curve(y_test, predicted)

    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'MLA used'] = MLA_name
    MLA_compare.loc[row_index, 'Train Accuracy'] = round(alg.score(train_embeddings,y_train), 4)
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(alg.score(test_embeddings, y_test), 4)
    
    
    recall = recall_score(test['target'], predicted)
    precision = precision_score(test['target'], predicted)
    MLA_compare.loc[row_index, 'Precission'] = precision
    MLA_compare.loc[row_index, 'Recall'] = recall
    MLA_compare.loc[row_index, 'F1-score'] = round((2*precision*recall)/(precision+recall),4)
    MLA_compare.loc[row_index, 'AUC'] = auc(fp, tp)

    row_index+=1

In [ ]:
# f1-score 기준 정렬
MLA_compare.sort_values(by = ['F1-score'], ascending = False, inplace = True)    
MLA_compare

# Comapring all MLA
## wordCount
- precision
- recall
- accuracy
- f1-score

In [ ]:
# Comparing all machine learning algorithms
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve

row_index = 0
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)

for alg in MLA:
    predicted = alg.fit(X_train, y_train).predict(X_test)
    fp, tp, th = roc_curve(y_test, predicted)

    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'MLA used'] = MLA_name
    MLA_compare.loc[row_index, 'Train Accuracy'] = round(alg.score(X_train, y_train), 4)
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(alg.score(X_test, y_test), 4)
    
    
    recall = recall_score(y_test, predicted)
    precision = precision_score(y_test, predicted)
    MLA_compare.loc[row_index, 'Precission'] = precision
    MLA_compare.loc[row_index, 'Recall'] = recall
    MLA_compare.loc[row_index, 'F1-score'] = round((2*precision*recall)/(precision+recall),4)
    MLA_compare.loc[row_index, 'AUC'] = auc(fp, tp)

    row_index+=1

In [ ]:
# f1-score 기준 정렬
MLA_compare.sort_values(by = ['F1-score'], ascending = False, inplace = True)    
MLA_compare